In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras


print(tf.test.is_gpu_available())
print(tf.__version__)
print(sys.version_info)

for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

True
2.0.0
sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)
matplotlib 3.2.2
numpy 1.18.5
pandas 1.0.5
sklearn 0.21.2
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [3]:
from sklearn.model_selection import train_test_split

x_train_all,x_test,y_train_all,y_test = train_test_split(housing.data,housing.target,random_state=7)

x_train,x_valid,y_train,y_vaild=train_test_split(x_train_all,y_train_all,random_state=11)

print(x_train.shape,y_train.shape)
print(x_valid.shape,y_vaild.shape)
print(x_test.shape,y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [4]:
# 数据标准化

from sklearn.preprocessing import StandardScaler

saclar = StandardScaler()
x_train_sacled = saclar.fit_transform(x_train)
x_valid_sacled = saclar.transform(x_valid)
x_test_sacled=saclar.transform(x_test)

In [5]:
# 生成CSV文件
output_dir = "generate_csv"

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

def save_to_csv(output_dir,data,name_prefix,header=None,n_parts=10):
    path_format=os.path.join(output_dir,"{}_{:02d}.csv")
    filenames = []
    
    for file_idx, row_indices in enumerate(
                                np.array_split(np.arange(len(data)),n_parts)):
        print(name_prefix,file_idx)
        part_csv = path_format.format(name_prefix,file_idx)
        filenames.append(part_csv)
        with open(part_csv,"wt",encoding="utf-8") as f:
            if header is not None:
                f.write(header+"\n")
            for row_index in row_indices:
                f.write(",".join(
                    [repr(col) for col in data[row_index]]))
                f.write("\n")
    return filenames

# merge合并数据
train_data = np.c_[x_train_sacled,y_train]
valid_data = np.c_[x_valid_sacled,y_vaild]
test_data = np.c_[x_test_sacled,y_test]

header_cols = housing.feature_names+["MidianHouseValue"]
header_str = ",".join(header_cols)

train_filenames = save_to_csv(output_dir,train_data,"train",header_str,n_parts=20)
valid_filenames = save_to_csv(output_dir,valid_data,"valid",header_str,n_parts=10)
test_filenames = save_to_csv(output_dir,test_data,"test",header_str,n_parts=10)

train 0
train 1
train 2
train 3
train 4
train 5
train 6
train 7
train 8
train 9
train 10
train 11
train 12
train 13
train 14
train 15
train 16
train 17
train 18
train 19
valid 0
valid 1
valid 2
valid 3
valid 4
valid 5
valid 6
valid 7
valid 8
valid 9
test 0
test 1
test 2
test 3
test 4
test 5
test 6
test 7
test 8
test 9


In [6]:
# 读取CSV文件

import pprint
print("train filenames:")
pprint.pprint(train_filenames)
print("valid filenames:")
pprint.pprint(valid_filenames)
print("test filenames:")
pprint.pprint(test_filenames)


train filenames:
['generate_csv\\train_00.csv',
 'generate_csv\\train_01.csv',
 'generate_csv\\train_02.csv',
 'generate_csv\\train_03.csv',
 'generate_csv\\train_04.csv',
 'generate_csv\\train_05.csv',
 'generate_csv\\train_06.csv',
 'generate_csv\\train_07.csv',
 'generate_csv\\train_08.csv',
 'generate_csv\\train_09.csv',
 'generate_csv\\train_10.csv',
 'generate_csv\\train_11.csv',
 'generate_csv\\train_12.csv',
 'generate_csv\\train_13.csv',
 'generate_csv\\train_14.csv',
 'generate_csv\\train_15.csv',
 'generate_csv\\train_16.csv',
 'generate_csv\\train_17.csv',
 'generate_csv\\train_18.csv',
 'generate_csv\\train_19.csv']
valid filenames:
['generate_csv\\valid_00.csv',
 'generate_csv\\valid_01.csv',
 'generate_csv\\valid_02.csv',
 'generate_csv\\valid_03.csv',
 'generate_csv\\valid_04.csv',
 'generate_csv\\valid_05.csv',
 'generate_csv\\valid_06.csv',
 'generate_csv\\valid_07.csv',
 'generate_csv\\valid_08.csv',
 'generate_csv\\valid_09.csv']
test filenames:
['generate_csv\\test

In [7]:
#  1. filename-> dataset
#  2. read file ->dataset -> datasets -> merge
#  3. parse csv
filename_dataset = tf.data.Dataset.list_files(train_filenames)
for item in filename_dataset:
    print(item)
    
n_readers = 5
dataset = filename_dataset.interleave(
        # header不要，跳过
        lambda filename : tf.data.TextLineDataset(filename).skip(1),
        cycle_length = n_readers
)
for line in dataset.take(10):
    print(line)

tf.Tensor(b'generate_csv\\train_17.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_09.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_01.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_07.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_13.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_10.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_08.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_14.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_15.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_05.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_16.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_00.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_06.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_02.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_19.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\

In [8]:
# tf.io.decode_scv(str, record_defaults)
# 把一个字符串转换成Tensor列表
sample_str ="1,2,3,4,5"
record_defaults = [tf.constant(0, dtype=tf.int32)] * 5
parsed_fields = tf.io.decode_csv(sample_str,record_defaults)
print(parsed_fields)

[<tf.Tensor: id=91, shape=(), dtype=int32, numpy=1>, <tf.Tensor: id=92, shape=(), dtype=int32, numpy=2>, <tf.Tensor: id=93, shape=(), dtype=int32, numpy=3>, <tf.Tensor: id=94, shape=(), dtype=int32, numpy=4>, <tf.Tensor: id=95, shape=(), dtype=int32, numpy=5>]


In [9]:
sample_str ="1,2,3,4,5"
record_defaults = [
    tf.constant(0, dtype=tf.int32),
    0,
    np.nan,
    "hello",
    tf.constant([])
] 
parsed_fields = tf.io.decode_csv(sample_str,record_defaults)
print(parsed_fields)

[<tf.Tensor: id=102, shape=(), dtype=int32, numpy=1>, <tf.Tensor: id=103, shape=(), dtype=int32, numpy=2>, <tf.Tensor: id=104, shape=(), dtype=float32, numpy=3.0>, <tf.Tensor: id=105, shape=(), dtype=string, numpy=b'4'>, <tf.Tensor: id=106, shape=(), dtype=float32, numpy=5.0>]


In [10]:
try:
    tf.io.decode_csv(",,,,",record_defaults)
except Exception as ex:
    print(ex)

Field 4 is required but missing in record 0! [Op:DecodeCSV]


In [11]:
try:
    tf.io.decode_csv("1,2,3,4,5,6,7",record_defaults)
except Exception as ex:
    print(ex)

Expect 5 fields but have 7 in record 0 [Op:DecodeCSV]


In [12]:
def parse_csv_line(line,n_fields=9):
    defs = [tf.constant(np.nan)] * n_fields
    parsed_fields = tf.io.decode_csv(line,record_defaults=defs)
    # 特征值和目标值转换成向量     
    x = tf.stack(parsed_fields[0:-1])
    y = tf.stack(parsed_fields[-1:])
    return x,y
    
parse_csv_line(b'-0.8219588176953616,1.874166156711919,0.18212349433218608,-0.03170019246279883,-0.6011178900722581,-0.14337494105109344,1.0852205298015787,-0.8613994495208361,1.054'
              ,n_fields=9)

(<tf.Tensor: id=126, shape=(8,), dtype=float32, numpy=
 array([-0.82195884,  1.8741661 ,  0.1821235 , -0.03170019, -0.6011179 ,
        -0.14337493,  1.0852206 , -0.8613995 ], dtype=float32)>,
 <tf.Tensor: id=127, shape=(1,), dtype=float32, numpy=array([1.054], dtype=float32)>)

In [13]:
#  1. filename-> dataset
#  2. read file ->dataset -> datasets -> merge
#  3. parse csv

def csv_reader_dataset(filenames,n_readers=5,batch_size=32,n_parse_theads=5,shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat()
    dataset = dataset.interleave(
        lambda filename : tf.data.TextLineDataset(filename).skip(1),
        cycle_length=n_readers
    )
    dataset.shuffle(shuffle_buffer_size)
    dataset=dataset.map(parse_csv_line,num_parallel_calls = n_parse_theads)
    dataset = dataset.batch(batch_size)
    return dataset

train_set = csv_reader_dataset(train_filenames,batch_size=3)
print(train_set)
for x_batch,y_batch in train_set.take(2):
    print("x:")
    pprint.pprint(x_batch)
    print("y:")
    pprint.pprint(y_batch)

<DatasetV1Adapter shapes: ((None, 8), (None, 1)), types: (tf.float32, tf.float32)>
x:
<tf.Tensor: id=211, shape=(3, 8), dtype=float32, numpy=
array([[ 0.4240821 ,  0.91296333, -0.04437482, -0.15297213, -0.24727628,
        -0.10539167,  0.86126745, -1.335779  ],
       [ 0.15782312,  0.4323619 ,  0.3379948 , -0.01588031, -0.37338907,
        -0.05305246,  0.80061346, -1.2359096 ],
       [-1.1157656 ,  0.99306357, -0.334192  , -0.06535219, -0.32893205,
         0.04343066, -0.12785879,  0.30707204]], dtype=float32)>
y:
<tf.Tensor: id=212, shape=(3, 1), dtype=float32, numpy=
array([[3.955],
       [3.169],
       [0.524]], dtype=float32)>
x:
<tf.Tensor: id=213, shape=(3, 8), dtype=float32, numpy=
array([[ 0.48530516, -0.8492419 , -0.06530126, -0.02337966,  1.4974351 ,
        -0.07790658, -0.90236324,  0.78145146],
       [-1.119975  , -1.3298433 ,  0.14190045,  0.4658137 , -0.10301778,
        -0.10744184, -0.7950524 ,  1.5304717 ],
       [ 0.4369235 , -1.9706452 , -0.16642106,  0.054

In [14]:
batch_size = 32
train_set = csv_reader_dataset(train_filenames,batch_size=batch_size)
valid_set = csv_reader_dataset(valid_filenames,batch_size=batch_size)
test_set = csv_reader_dataset(test_filenames,batch_size=batch_size)

In [15]:

model = keras.models.Sequential([
 
    keras.layers.Dense(30,activation="relu",input_shape=[8]),
    keras.layers.Dense(1),
])
model.summary()
# 目标函数：均方差，optimizer：随机梯度下降
model.compile(loss="mean_squared_error",optimizer="sgd")
callbacks=[keras.callbacks.EarlyStopping(patience=5,min_delta=1e-3)]
history = model.fit(train_set,
                    validation_data=valid_set,
                    steps_per_epoch = 11160 // batch_size,
                    validation_steps = 3870 // batch_size, 
                    epochs=100,
                    callbacks=callbacks)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                270       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________
Train for 348 steps, validate for 120 steps
Epoch 1/100
348/348 [==============================] - 2s 5ms/step - loss: 1.0282 - val_loss: 1.4354
Epoch 2/100
348/348 [==============================] - 1s 2ms/step - loss: 1.0075 - val_loss: 0.5657
Epoch 3/100
348/348 [==============================] - 1s 2ms/step - loss: 0.4736 - val_loss: 0.4721
Epoch 4/100
348/348 [==============================] - 1s 2ms/step - loss: 0.4276 - val_loss: 0.4335
Epoch 5/100
348/348 [==============================] - 1s 2ms/step - los

In [17]:
model.evaluate(test_set,steps = 5160 // batch_size)

161/161 [==============================] - 0s 1ms/step - loss: 0.3559


0.35589015187684053